# Calculate distance from cities in UCDB dataset

1. Distance to shoreline (shoreline is the OCEAN MASK dataset from the official World Bank boundaries)
2. Distance to border (from WB GAD)

In [1]:
import sys
import os
import itertools
import rasterio

import geopandas as gpd
import pandas as pd
import numpy as np

from scipy.spatial import cKDTree
from shapely.geometry import Point, LineString
from operator import itemgetter

sys.path.insert(0, "C:/WBG/Work/Code/GOSTrocks/src")

import GOSTrocks.ntlMisc as ntlMisc
import GOSTrocks.rasterMisc as rMisc
import GOSTrocks.shapeMisc as shapeMisc
import GOSTrocks.misc as misc
from GOSTrocks.misc import tPrint

%load_ext autoreload
%autoreload 2

In [2]:
data_folder = r"C:\WBG\Work\data\URBAN"

ucdb_file = os.path.join(data_folder, "GHS_UCDB_GLOBE_R2024A.gpkg")
# fua_file = os.path.join(data_folder, "GHS_FUA_UCDB2015_GLOBE_R2019A_54009_1K_V1_0.gpkg")
# fua_peripheries = os.path.join(data_folder, "FUA_peripheries.gpkg")
# oxford_extents = os.path.join(data_folder, "OE_FUA_SHAPEFILE", "OE_FUA_SHAPEFILE.shp")

ucdb_id = "ID_UC_G0"

coastline_file = r"C:\WBG\Work\data\ADMIN\BETTER_FORMATS\ocean_mask.gpkg"
border_file = r"C:\WBG\Work\data\ADMIN\BETTER_FORMATS\WB_GAD_Lines.gpkg"

out_folder = r"C:\WBG\Work\Projects\MENA_Urban\RESULTS\DISTANCE"
if not os.path.exists(out_folder):
    os.makedirs(out_folder)

In [3]:
inU = gpd.read_file(ucdb_file)
inU = inU.loc[:, [ucdb_id, "geometry"]]
inU.head()

c:\wbg\Anaconda3\envs\s2s_ingest\lib\site-packages\pyogrio\geopandas.py:265: UserWarning: More than one layer found in 'GHS_UCDB_GLOBE_R2024A.gpkg': 'GHS_UCDB_THEME_CLIMATE_GLOBE_R2024A' (default), 'GHS_UCDB_THEME_EXPOSURE_GLOBE_R2024A', 'GHS_UCDB_THEME_GENERAL_CHARACTERISTICS_GLOBE_R2024A', 'GHS_UCDB_THEME_GEOGRAPHY_GLOBE_R2024A', 'GHS_UCDB_THEME_GHSL_GLOBE_R2024A', 'GHS_UCDB_THEME_GREENNESS_GLOBE_R2024A', 'GHS_UCDB_THEME_HAZARD_RISK_GLOBE_R2024A', 'GHS_UCDB_THEME_INFRASTRUCTURES_GLOBE_R2024A', 'GHS_UCDB_THEME_LULC_GLOBE_R2024A', 'GHS_UCDB_THEME_NATURAL_SYSTEMS_GLOBE_R2024A', 'GHS_UCDB_THEME_SDG_GLOBE_R2024A', 'GHS_UCDB_THEME_SOCIOECONOMIC_GLOBE_R2024A', 'GHS_UCDB_THEME_WATER_GLOBE_R2024A', 'GHS_UCDB_THEME_EMISSIONS_GLOBE_R2024A', 'GHS_UCDB_THEME_HEALTH_GLOBE_R2024A', 'UC_centroids'. Specify layer parameter to avoid this warning.
  result = read_func(


,ID_UC_G0,geometry
0,1,"MULTIPOLYGON (((-16906000 -1703000, -16905000 ..."
1,2,"MULTIPOLYGON (((-16819000 -2590000, -16817000 ..."
2,3,"MULTIPOLYGON (((-15157000 2615000, -15156000 2..."
3,4,"MULTIPOLYGON (((-14552000 -2155000, -14551000 ..."
4,5,"MULTIPOLYGON (((-10562000 3925000, -10560000 3..."


In [4]:
inCoasts = gpd.read_file(coastline_file)
#inCoasts['geometry'] = inCoasts.geometry.buffer(0)
inCoasts = misc.explodeGDF(inCoasts)
inCoasts['geometry'] = inCoasts['geometry'].boundary
inCoasts.drop(['level_0', 'index', 'id'], inplace=True, axis=1, errors='ignore')
inCoasts = misc.explodeGDF(inCoasts)
inCoasts.drop(['level_0', 'index', 'id'], inplace=True, axis=1, errors='ignore')
inCoasts['length'] = inCoasts.geometry.length
inCoasts = inCoasts.iloc[1:]
inCoasts = inCoasts.to_crs(inU.crs)
inCoasts.to_file(os.path.join(out_folder, "coastlines.gpkg"), driver="GPKG")


C:\Users\WB411133\AppData\Local\Temp\ipykernel_18144\334923102.py:8: UserWarning: Geometry is in a geographic CRS. Results from 'length' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  inCoasts['length'] = inCoasts.geometry.length


In [5]:
inBorders = gpd.read_file(border_file)
inBorders.explode(index_parts=True, ignore_index=True, inplace=True)
inBorders['ID'] = list(range(0, len(inBorders)))
inBorders = inBorders.to_crs(inU.crs)
inBorders = misc.explodeGDF(inBorders)
inBorders.drop(['level_0', 'index'], inplace=True, axis=1, errors='ignore')

In [6]:
tPrint("Started")
inU_borders = gpd.sjoin_nearest(inU, inBorders, how="left", distance_col="BORDER_DIST")
tPrint("Finished border distances")
inU_coasts = gpd.sjoin_nearest(inU, inCoasts, how="left", distance_col="COAST_DIST")
tPrint("Finished coastline distances")


15:47:48	Started
15:48:10	Finished border distances
16:04:01	Finished coastline distances


In [8]:
inU_combined = pd.merge(inU_borders, inU_coasts[[ucdb_id, "COAST_DIST"]], on=ucdb_id)
inU_combined.to_file(os.path.join(out_folder, "UCDB_distances.gpkg"), driver="GPKG", index=False)
inU_combined[['ID_UC_G0', 'BORDER_DIST', 'COAST_DIST']].to_csv(os.path.join(out_folder, "UCDB_distances.csv"), index=False)